In [1]:
!nvidia-smi -L           # should show a T4 or P100
from google.colab import drive
drive.mount("/content/drive")

GPU 0: Tesla T4 (UUID: GPU-ee42a602-7e8d-6558-7036-cc22787bbf3e)
Mounted at /content/drive


In [12]:
#@title 2. Install Kaggle API
!pip -q install kaggle --upgrade

# If you haven't uploaded kaggle.json yet, run this once:
from google.colab import files
files.upload()   # choose your kaggle.json

# Move token to the right place
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/ 2>/dev/null || true
!chmod 600 ~/.kaggle/kaggle.json

# Quick sanity-check
!kaggle datasets list -s "gold price" | head


Saving kaggle.json to kaggle (2).json
ref                                                              title                                                  size  lastUpdated                 downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  -----------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
sid321axn/gold-price-prediction-dataset                          Gold Price Prediction Dataset                        379629  2021-07-20 14:33:47.593000          20907        162  0.9411765        
franciscogcc/financial-data                                      Gold Price Regression                                522571  2024-10-23 23:35:36.087000           4393         46  1.0              
tunguz/gold-prices                                               Gold Prices                                            5555  2021-03-30 13:16:56.250000           9506   

In [17]:
!pip -q install yfinance pandas_datareader --upgrade

In [25]:
# 2. Imports
import yfinance as yf, pandas as pd, pathlib
from pandas_datareader import data as pdr

# 3. Fetch Yahoo Finance time-series
start = "2010-01-01"

gold = yf.download("GC=F",   start=start, auto_adjust=False)["Close"]
gold.name = "Gold"

usd  = yf.download("DX-Y.NYB", start=start, auto_adjust=False)["Close"]
usd.name = "USD"

oil  = yf.download("CL=F",   start=start, auto_adjust=False)["Close"]
oil.name = "Oil"



# 4. Fetch CPI from FRED
cpi  = (pdr.DataReader("CPIAUCSL", "fred", start)
          .resample("D").ffill().rename(columns={"CPIAUCSL":"CPI"}))

# 5. Merge & clean
df = pd.concat([gold, usd, oil, cpi], axis=1).dropna()

# 6. Save
out = "/content/drive/MyDrive/raw.csv"
pathlib.Path(out).parent.mkdir(parents=True, exist_ok=True)
df.to_csv(out)
print("Saved", out, "shape:", df.shape)
df.tail()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Saved /content/drive/MyDrive/raw.csv shape: (3854, 4)


,GC=F,DX-Y.NYB,CL=F,CPI
2025-04-25,3282.399902,99.470001,63.020000,320.321
2025-04-28,3332.500000,99.010002,62.049999,320.321
2025-04-29,3318.800049,99.239998,60.419998,320.321
2025-04-30,3305.000000,99.470001,58.209999,320.321
2025-05-01,3210.000000,100.250000,59.240002,320.580
